<h2> Absolutely exploratory analyses on the 0.5deg res results</h2>
  - Goals are to check compatibility
  - Also to check specifications of the regridding file

In [1]:
import pandas as pd

In [5]:
h5df = pd.read_hdf('./results-xgboost-tmp2m-highres.h5', 'train')
h5df.reset_index(inplace=True)

In [6]:
print(h5df.start_date.min())
print(h5df.start_date.max())

2013-12-04 00:00:00
2020-10-06 00:00:00


In [18]:
h5df

,start_date,lat,lon,xgboost_tmp2m_pred_zscore,tmp2m_daily_mean,tmp2m_daily_std,tmp2m_zscore,tmp2m,xgboost_tmp2m_pred
0,2013-12-04,27.25,260.75,-0.444575,14.477173,2.051871,-0.306895,13.847465,13.564963
1,2013-12-04,27.25,261.25,-0.399303,14.199499,1.951358,-0.225235,13.759985,13.420315
2,2013-12-04,27.25,261.75,-0.429326,14.463642,1.931331,-0.141234,14.190873,13.634471
3,2013-12-04,27.25,262.25,-0.425091,14.828850,1.963461,-0.109792,14.613278,13.994200
4,2013-12-04,27.25,277.75,0.390466,17.885119,2.932417,0.858304,20.402023,19.030128
...,...,...,...,...,...,...,...,...,...
7764388,2020-10-06,47.75,270.25,-2.606435,4.096629,2.030455,-2.627814,-1.239028,-1.195618
7764389,2020-10-06,47.75,270.75,-2.271173,4.436356,1.982421,-2.204216,0.066671,-0.066067
7764390,2020-10-06,47.75,271.25,-2.178175,4.620783,1.995103,-2.001210,0.628163,0.275098
7764391,2020-10-06,47.75,271.75,-2.075516,4.778328,1.958868,-2.032807,0.796327,0.712666
